In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Physical_Exercise_Response/GSE111552'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Blood and skeletal muscle gene signatures of exercise training in men (PBMC data set)"
!Series_summary	"Accounting for transcriptional features of endurance adaptations to training is important and could help elucidate the high variability in oxygen uptake (VO2) response. We aimed to identify whole-transcriptome signatures of an endurance training protocol in whole-blood (leukocytes), PBMCs and skeletal muscle tissue of the same group of individuals in a controlled environment."
!Series_overall_design	"Two groups of healthy untrained policemen recruits (G1, n=20, 26±4 yrs; G2, n= 21, 27±3 yrs) were selected. Blood (PAXgene and EDTA tubes) and muscle tissueforgene expression assays and peak VO2peak measures by cardiopulmonary exercise test were taken before and after 18 weeks of running (3 times/wk; 60min). RNA was extracted from whole-blood (leukocytes), PBMCs and muscle and was used for whole-genome expression analysis using AffymetrixGene Chip H

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
trait_row = 5  # Mapping Physical_Exercise_Response to key holding VO2peak (l/min) values
age_row = 2  # Mapping age to key holding age values
gender_row = 1  # Mapping gender to key holding gender values

# 2.3 Data Type Conversion

def convert_trait(value):
    try:
        return float(value.split(": ")[1])
    except:
        return None

def convert_age(value):
    try:
        return float(value.split(": ")[1])
    except:
        return None

def convert_gender(value):
    try:
        return 1 if value.split(": ")[1].lower() == 'male' else 0
    except:
        return None

# 3. Save Metadata
save_cohort_info('GSE111552', './preprocessed/Physical_Exercise_Response/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Physical_Exercise_Response', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Physical_Exercise_Response/trait_data/GSE111552.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM3034063': [4.127, 27.0, 1.0], 'GSM3034064': [5.23, 19.0, 1.0], 'GSM3034065': [3.968, 21.0, 1.0], 'GSM3034066': [3.568, 30.0, 1.0], 'GSM3034067': [3.28, 22.0, 1.0], 'GSM3034068': [3.843, 21.0, 1.0], 'GSM3034069': [4.559, 27.0, 1.0], 'GSM3034070': [3.144, 29.0, 1.0], 'GSM3034071': [4.379, 28.0, 1.0], 'GSM3034072': [3.249, 28.0, 1.0], 'GSM3034073': [3.802, 28.0, 1.0], 'GSM3034074': [2.87, 27.0, 1.0], 'GSM3034075': [3.691, 30.0, 1.0], 'GSM3034076': [4.925, 22.0, 1.0], 'GSM3034077': [3.069, 29.0, 1.0], 'GSM3034078': [3.152, 25.0, 1.0], 'GSM3034079': [4.571, 27.0, 1.0], 'GSM3034080': [5.328, 19.0, 1.0], 'GSM3034081': [4.045, 21.0, 1.0], 'GSM3034082': [3.849, 30.0, 1.0], 'GSM3034083': [4.06, 22.0, 1.0], 'GSM3034084': [4.39, 21.0, 1.0], 'GSM3034085': [5.191, 27.0, 1.0], 'GSM3034086': [3.791, 29.0, 1.0], 'GSM3034087': [5.271, 28.0, 1.0], 'GSM3034088': [3.778, 28.0, 1.0], 'GSM3034089': [4.073, 28.0, 1.0], 'GSM3034090': [3.444, 27.0, 1.0], 'GSM3034091': [4.473, 30.0, 1.0], 'GSM3034092': [5.0

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:

# 1. Determine the appropriate keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function
probe_to_gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function to get the gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, probe_to_gene_mapping)
